In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/lmsys-chatbot-arena/sample_submission.csv
/kaggle/input/lmsys-chatbot-arena/train.csv
/kaggle/input/lmsys-chatbot-arena/test.csv


In [2]:
import os
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import regex as re

from sklearn.feature_extraction.text import TfidfVectorizer

from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, f1_score, recall_score, precision_score, accuracy_score, roc_auc_score, log_loss
from sklearn.preprocessing import LabelEncoder
import nltk
from nltk.corpus import stopwords, wordnet
from nltk.stem import PorterStemmer
import string
from nltk.stem import WordNetLemmatizer
import torch
from torchtext.data.utils import get_tokenizer
from torchtext.vocab import build_vocab_from_iterator
from torch.nn.utils.rnn import pad_sequence
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
import torch.nn as nn
from torch.utils.data import DataLoader, Dataset
import torch
import torch.nn as nn
import torch.optim as optim
from torch.nn.utils.rnn import pad_sequence


In [3]:
# Define the path to the downloaded NLTK data
nltk_data_path = '/path/to/your/nltk_data'

# Tell NLTK to look for data in the specified directory
nltk.data.path.append(nltk_data_path)

# Load the necessary resources
stop_words = set(stopwords.words('english'))
stemmer = PorterStemmer()
lemmatizer = WordNetLemmatizer()


In [4]:
train = pd.read_csv("/kaggle/input/lmsys-chatbot-arena/train.csv")

test = pd.read_csv("/kaggle/input/lmsys-chatbot-arena/test.csv")

In [5]:
train

,id,model_a,model_b,prompt,response_a,response_b,winner_model_a,winner_model_b,winner_tie
0,30192,gpt-4-1106-preview,gpt-4-0613,"[""Is it morally right to try to have a certain...","[""The question of whether it is morally right ...","[""As an AI, I don't have personal beliefs or o...",1,0,0
1,53567,koala-13b,gpt-4-0613,"[""What is the difference between marriage lice...","[""A marriage license is a legal document that ...","[""A marriage license and a marriage certificat...",0,1,0
2,65089,gpt-3.5-turbo-0613,mistral-medium,"[""explain function calling. how would you call...","[""Function calling is the process of invoking ...","[""Function calling is the process of invoking ...",0,0,1
3,96401,llama-2-13b-chat,mistral-7b-instruct,"[""How can I create a test set for a very rare ...","[""Creating a test set for a very rare category...","[""When building a classifier for a very rare c...",1,0,0
4,198779,koala-13b,gpt-3.5-turbo-0314,"[""What is the best way to travel from Tel-Aviv...","[""The best way to travel from Tel Aviv to Jeru...","[""The best way to travel from Tel-Aviv to Jeru...",0,1,0
...,...,...,...,...,...,...,...,...,...
57472,4294656694,gpt-4-0613,claude-1,"[""A simple mnemonic for \u03c0:\n\""How I wish ...","[""Sure, let's break it down:\n\n1. \""How\"" has...","[""Here is how that mnemonic represents the dig...",1,0,0
57473,4294692063,claude-2.0,llama-2-13b-chat,"[""In python, implement a naive Bayes with gaus...","[""Here is an implementation of a naive Bayes c...","[""Sure! Here's an implementation of a naive Ba...",1,0,0
57474,4294710549,claude-1,alpaca-13b,"[""is it unethical to work on building weapons?...","[""Working on weapons technology raises some et...","[""It depends on the context. Weapons can be us...",1,0,0
57475,4294899228,palm-2,tulu-2-dpo-70b,"[""If a bait contains 0,0025% bromadiolon then ...","[""Bromadiolone is a rodenticide which is most ...","[""As an AI language model, I do not promote or...",0,1,0


In [6]:
target_columns = ['winner_model_a', 'winner_model_b', 'winner_tie']

In [7]:
y = train[target_columns].idxmax(axis=1) 

# Encode labels
label_encoder = LabelEncoder()
y_encoded = label_encoder.fit_transform(y)

In [8]:
labels = torch.tensor(y_encoded, dtype=torch.int64)

In [9]:
labels

tensor([0, 1, 2,  ..., 0, 1, 0])

# Data Preprocessing

## Clean

In [10]:
def clean_text(text):
    # Replace escaped characters with their actual representations
    text = text.replace(r'\"', '"')
    text = text.replace(r"\'", "'")
    text = text.replace(r"\\n", " ")
    text = text.replace(r"\\/\\/", "//")
    
    # Remove code block markers
    text = re.sub(r'```.*?```', '', text, flags=re.DOTALL)
    
    # Convert to lowercase
    text = text.lower()
    
    # Remove punctuation
    text = text.translate(str.maketrans('', '', string.punctuation))
    
    # Normalize whitespace
    text = re.sub(r'\s+', ' ', text).strip()
    
    return text


In [11]:
X_train = pd.DataFrame()
X_test=pd.DataFrame()

In [12]:
X_train["question"]=train["prompt"].apply(clean_text)
X_train["response_a"]=train["response_a"].apply(clean_text)
X_train["response_b"]=train["response_b"].apply(clean_text)


In [13]:
X_test["question"]=test["prompt"].apply(clean_text)
X_test["response_a"]=test["response_a"].apply(clean_text)
X_test["response_b"]=test["response_b"].apply(clean_text)


## Tokenizer and vocabulary

In [14]:
# Tokenizer
tokenizer = get_tokenizer('basic_english')

# Function to yield tokens from data iterator
def yield_tokens(data_iter):
    for text in data_iter:
        yield tokenizer(text)

# Build vocabulary from training data
vocab = build_vocab_from_iterator(yield_tokens(X_train['question'] + X_train['response_a'] + X_train['response_b']), specials=["<unk>"])
vocab.set_default_index(vocab['<unk>'])


def text_pipeline(text):
    return vocab(tokenizer(text))


In [15]:
question_list = [torch.tensor(text_pipeline(text), dtype=torch.int64) for text in X_train['question']]
answer1_list = [torch.tensor(text_pipeline(text), dtype=torch.int64) for text in X_train['response_a']]
answer2_list = [torch.tensor(text_pipeline(text), dtype=torch.int64) for text in X_train['response_b']]

# Pad sequences
train_padded_questions = pad_sequence(question_list, batch_first=True)
train_padded_answera = pad_sequence(answer1_list, batch_first=True)
train_padded_answerb = pad_sequence(answer2_list, batch_first=True)

In [16]:
test_question_list = [torch.tensor(text_pipeline(text), dtype=torch.int64) for text in X_test['question']]
test_answer1_list = [torch.tensor(text_pipeline(text), dtype=torch.int64) for text in X_test['response_a']]
test_answer2_list = [torch.tensor(text_pipeline(text), dtype=torch.int64) for text in X_test['response_b']]

# Pad sequences
test_padded_questions = pad_sequence(test_question_list, batch_first=True)
test_padded_answera = pad_sequence(test_answer1_list, batch_first=True)
test_padded_answerb = pad_sequence(test_answer2_list, batch_first=True)

In [17]:
from torch.utils.data import DataLoader, TensorDataset


In [18]:
batch_size=70
# Create DataLoader for training data
train_dataset = TensorDataset(train_padded_questions, train_padded_answera, train_padded_answerb, labels)
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)

In [19]:
test_dataset = TensorDataset(test_padded_questions, test_padded_answera, test_padded_answerb)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

## MODEL

In [20]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')


In [21]:
import torch.nn.functional as F

class RNNModel(nn.Module):
    def __init__(self, vocab_size, embed_dim, hidden_dim, output_dim):
        super(RNNModel, self).__init__()
        self.embedding = nn.Embedding(vocab_size, embed_dim)
        self.rnn = nn.LSTM(embed_dim, hidden_dim, batch_first=True)
        self.fc = nn.Linear(hidden_dim * 3, output_dim)  # Concatenate question and both answers

    def forward(self, question, answer1, answer2):
        question_emb = self.embedding(question)
        answer1_emb = self.embedding(answer1)
        answer2_emb = self.embedding(answer2)

        _, (hidden_q, _) = self.rnn(question_emb)
        _, (hidden_a1, _) = self.rnn(answer1_emb)
        _, (hidden_a2, _) = self.rnn(answer2_emb)

        concatenated = torch.cat((hidden_q.squeeze(0), hidden_a1.squeeze(0), hidden_a2.squeeze(0)), dim=1)
        output = self.fc(concatenated)

        # Apply softmax to get probabilities
        probs = nn.functional.softmax(output, dim=1)

        return probs

In [22]:
vocab_size = len(vocab)
embed_dim = 100
hidden_dim = 128
output_dim = 3  # Three probabilities

In [23]:
model = RNNModel(vocab_size, embed_dim, hidden_dim, output_dim).to(device)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

In [24]:
num_epochs = 3
for epoch in range(num_epochs):
    model.train()  # Set the model to training mode
    total_loss = 0
    total_correct = 0
    for batch in train_loader:
        questions, answers1, answers2, labels = batch
        questions = pad_sequence(questions, batch_first=True).to(device)
        answers1 = pad_sequence(answers1, batch_first=True).to(device)
        answers2 = pad_sequence(answers2, batch_first=True).to(device)
        labels = labels.to(device)

        optimizer.zero_grad()
        outputs = model(questions, answers1, answers2)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        total_loss += loss.item()
        total_correct += (outputs.argmax(1) == labels).sum().item()

    avg_loss = total_loss / len(train_loader)
    accuracy = total_correct / len(train_loader.dataset)
    print(f'Epoch {epoch+1}/{num_epochs}, Loss: {avg_loss:.4f}, Accuracy: {accuracy:.4f}')


Epoch 1/3, Loss: 1.0977, Accuracy: 0.3474
Epoch 2/3, Loss: 1.0974, Accuracy: 0.3475
Epoch 3/3, Loss: 1.0973, Accuracy: 0.3474


In [25]:
model.eval()
test_predictions = []
with torch.no_grad():
    for batch in test_loader:
        questions, answers1, answers2 = batch
        questions = pad_sequence(questions, batch_first=True).to(device)
        answers1 = pad_sequence(answers1, batch_first=True).to(device)
        answers2 = pad_sequence(answers2, batch_first=True).to(device)

        outputs = model(questions, answers1, answers2)
        test_predictions.append(outputs)

print(test_predictions)

[tensor([[0.3614, 0.3558, 0.2828],
        [0.3452, 0.3658, 0.2890],
        [0.3412, 0.3455, 0.3132]], device='cuda:0')]


In [26]:
numpy_arrays = [tensor.cpu().numpy() for tensor in test_predictions]


In [27]:
# Convert list of numpy arrays to a single numpy array
combined_array = np.vstack(numpy_arrays)

In [28]:
combined_array

array([[0.36138615, 0.35584182, 0.2827721 ],
       [0.3452126 , 0.36576384, 0.28902358],
       [0.34124738, 0.3455378 , 0.31321484]], dtype=float32)

In [29]:
submission = pd.DataFrame({
    'id': test['id'],  # Include the 'id' column from the test dataset
    'winner_model_a': combined_array[:, 0],  # Probability predictions for winner_model_a
    'winner_model_b': combined_array[:, 1],  # Probability predictions for winner_model_b
    'winner_tie': combined_array[:, 2]  # Probability predictions for winner_tie
})

In [30]:
submission.head()

,id,winner_model_a,winner_model_b,winner_tie
0,136060,0.361386,0.355842,0.282772
1,211333,0.345213,0.365764,0.289024
2,1233961,0.341247,0.345538,0.313215


In [31]:
submission.to_csv('submission.csv', index=False)